In [ ]:
# STEP 1: Install required libraries
!pip install -q simpletransformers
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00


In [ ]:
# STEP 2: Import libraries
import pandas as pd
import logging
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import train_test_split

In [ ]:
!pip install -q scikit-learn
!pip install -q tensorflow

In [ ]:
# STEP 2: Imports
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [ ]:
# STEP 3: Mount Google Drive (if data is stored there)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# STEP 4: Load and prepare data
# Modify this path to where your OCR text files are stored
train_path = "/content/drive/MyDrive/meme_classifications/ocr_text/train/STask_C_train.csv"
eval_path = "/content/drive/MyDrive/meme_classifications/test_ocr_text/STask-C(index,text)test.csv"

In [ ]:
train_df = pd.read_csv(train_path)
eval_df = pd.read_csv(eval_path)

In [ ]:
# Confirm the structure
print(train_df.head())
print(eval_df.head())

       index                                               text  label
0  30003.png  gender envy from guys gender envy from cis gir...      0
1  30004.png  will things in the world ever get better for t...      1
2  30005.png  mgflip com gender identity gender expression c...      2
3  30007.png  ben shapiro character ai remember everything c...      1
4  30012.png  follow many cancelling disney to protect their...      2
       index                                               text
0  50048.png  schools d the military democrat cities woke ev...
1  50069.png  you want to support trans rights even though y...
2  50075.png                          hey i got you trans stuff
3  50160.png  hindendo nintendo of america intendoamerica we...
4  50181.png  when your date bails on you but your homie got...


In [ ]:
# STEP 5: Preprocessing for ALBERT
train_df = train_df.rename(columns={"text": "text", "label": "labels"})
train_data, val_data = train_test_split(train_df[["text", "labels"]], test_size=0.1, random_state=42)

In [ ]:
# STEP 6: ALBERT Model
model_args = ClassificationArgs()
model_args.num_train_epochs = 4
model_args.train_batch_size = 16
model_args.eval_batch_size = 32
model_args.max_seq_length = 128
model_args.output_dir = "outputs/subtaskC"
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.use_early_stopping = True
model_args.early_stopping_consider_epochs = True
model_args.early_stopping_delta = 0.01
model_args.early_stopping_patience = 2
model_args.logging_steps = 50

num_labels = train_df.labels.nunique()

albert_model = ClassificationModel(
    "albert",
    "albert-base-v2",
    num_labels=num_labels,
    args=model_args,
    use_cuda=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

In [ ]:
# Train ALBERT
albert_model.train_model(train_data, eval_df=val_data)

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:882: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 4:   0%|          | 0/228 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


...

Running Epoch 3 of 4:   0%|          | 0/228 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


(684,
 defaultdict(list,
             {'global_step': [228, 456, 684],
              'train_loss': [1.016526460647583,
               1.249436616897583,
               1.1744290590286255],
              'mcc': [0.0, 0.0, 0.0],
              'eval_loss': [1.0931602166249201,
               1.0943745466379018,
               1.0964042498515203]}))

In [ ]:
from transformers import AlbertTokenizer
import torch
from scipy.special import softmax
from tqdm import tqdm


In [ ]:
# STEP 8: Predict ALBERT on eval set — Get softmax probabilities from logits
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

inputs = tokenizer(
    eval_df["text"].tolist(),
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = albert_model.model.to(device)
model.eval()

input_ids = inputs["input_ids"].to(device)
attention_mask = inputs["attention_mask"].to(device)

with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# Convert to probabilities
albert_probs = softmax(logits.cpu().numpy(), axis=1)


In [ ]:
# STEP 7: ANN with TF-IDF features
tfidf = TfidfVectorizer(max_features=3000)
X_tfidf = tfidf.fit_transform(train_df["text"]).toarray()
X_eval_tfidf = tfidf.transform(eval_df["text"]).toarray()

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(train_df["labels"])
y_cat = to_categorical(y_encoded)

In [ ]:
# Build ANN model
ann_model = Sequential()
ann_model.add(Dense(256, activation='relu', input_shape=(X_tfidf.shape[1],)))
ann_model.add(Dropout(0.3))
ann_model.add(Dense(128, activation='relu'))
ann_model.add(Dropout(0.3))
ann_model.add(Dense(num_labels, activation='softmax'))

ann_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Train ANN
ann_model.fit(X_tfidf, y_cat, epochs=10, batch_size=32, validation_split=0.1, verbose=1)


Epoch 1/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.4006 - loss: 1.0781 - val_accuracy: 0.5210 - val_loss: 0.9966
Epoch 2/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6604 - loss: 0.8189 - val_accuracy: 0.4840 - val_loss: 1.0244
Epoch 3/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8291 - loss: 0.4891 - val_accuracy: 0.4864 - val_loss: 1.2048
Epoch 4/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9254 - loss: 0.2563 - val_accuracy: 0.4568 - val_loss: 1.5012
Epoch 5/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9687 - loss: 0.1314 - val_accuracy: 0.4469 - val_loss: 1.7491
Epoch 6/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9810 - loss: 0.0769 - val_accuracy: 0.4469 - val_loss: 1.9955
Epoch 7/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9862 - loss: 0.0585 - val_accuracy: 0.4494 - val_loss: 2.1449
Epoch 8/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9887 - loss: 0.0411 - val_accuracy: 0

In [ ]:
# Predict ANN on eval
ann_probs = ann_model.predict(X_eval_tfidf)
ann_preds = np.argmax(ann_probs, axis=1)

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step


In [ ]:
# STEP 8: Weighted Ensemble (ALBERT + ANN)
# Normalize predictions
albert_probs_norm = albert_probs / albert_probs.sum(axis=1, keepdims=True)
ann_probs_norm = ann_probs / ann_probs.sum(axis=1, keepdims=True)

In [ ]:
# Weighted average (adjust weights if needed)
ensemble_probs = 0.7 * albert_probs_norm + 0.3 * ann_probs_norm
ensemble_preds = np.argmax(ensemble_probs, axis=1)

In [ ]:
# Decode predictions if needed
final_labels = ensemble_preds

In [ ]:
import pandas as pd
import json

# Create the submission DataFrame
submission_df = pd.DataFrame({
    "index": eval_df["index"],
    "prediction": final_labels
})

# Convert DataFrame to list of dictionaries
submission_data = submission_df.to_dict(orient='records')

# Save to JSON
save_path = '/content/drive/MyDrive/SubTaskC.json'
with open(save_path, 'w') as json_file:
    json.dump(submission_data, json_file, indent=2)

print(f"✅ Submission file saved to: {save_path}")


✅ Submission file saved to: /content/drive/MyDrive/SubTaskC.json
